In [25]:
#%pip3 install opencv-python -q
#%pip3 install mediapipe -q 
#%pip3 install matplotlib
#%pip3 install numpy
#%pip3 install p

import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np 
import math 

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose 



In [26]:
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # imaging 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # variables 
        landmarks = results.pose_landmarks.landmark
        shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

        # rendering 
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=3, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=3, circle_radius=2) 
                                )               
        
        def display_parts(text, designation):
            cv2.putText(image, text, 
                        tuple(np.multiply(designation, [frame_width, frame_height]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
        def three_point_angle(a,b,c):
            a = np.array(a) # First
            b = np.array(b) # Mid
            c = np.array(c) # End
            
            radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
            angle = np.abs(radians*180.0/np.pi)
            
            if angle >180.0:
                angle = 360-angle
                
            return angle
        elbow_angle = three_point_angle(shoulder, elbow, wrist)
        
        def two_point_angle(p1, p2):
            opp = p2[0] - p1[0]
            adj = p2[1] - p1[1]
            radians = np.arctan(opp/adj)
            angle = radians*180/np.pi 
            
            return angle 
        shoulder_angle = two_point_angle(shoulder, elbow)
        
        display_parts("elbow", elbow)
        display_parts("shoulder", shoulder)
        display_parts("wrist", wrist)        
        display_parts(("       angle:"+str(elbow_angle)), elbow)
        display_parts(("             angle:"+str(shoulder_angle)), shoulder)
        
        
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        cv2.putText(image, 'SHOULDER', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(shoulder_angle), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        cv2.putText(image, 'ELBOW', (15,90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(elbow_angle), 
                    (10,140),        
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        # wrap-up 
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    


I0000 00:00:1703054109.447778       1 gl_context.cc:344] GL version: 2.1 (2.1 INTEL-20.6.4), renderer: Intel(R) UHD Graphics 630


AttributeError: 'NoneType' object has no attribute 'landmark'